In [1]:
import idaes.examples.workshops.Module_2_Flowsheet.hda_ideal_VLE as thermo_props
import idaes.examples.workshops.Module_2_Flowsheet.hda_reaction as reaction_props

from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.network import Arc, SequentialDecomposition

from idaes.core import FlowsheetBlock

from idaes.generic_models.unit_models import (Flash,
                               PressureChanger,
                               Mixer,
                               Separator as Splitter,
                               Heater,
                               StoichiometricReactor)
from idaes.generic_models.unit_models.pressure_changer import ThermodynamicAssumption

In [2]:
# Construct the model from idaes/examples/workshops/Module_2_Flowsheet/Module_2_Flowsheet_Solution.ipynb
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.thermo_params = thermo_props.HDAParameterBlock()
m.fs.reaction_params = reaction_props.HDAReactionParameterBlock(default={"property_package": m.fs.thermo_params})

m.fs.M101 = Mixer(default={"property_package": m.fs.thermo_params,
                           "inlet_list": ["toluene_feed", "hydrogen_feed", "vapor_recycle"]
                           })

m.fs.H101 = Heater(default={"property_package": m.fs.thermo_params,
                            "has_pressure_change": False,
                            "has_phase_equilibrium": True})
m.fs.R101 = StoichiometricReactor(default={"property_package": m.fs.thermo_params,
                                           "reaction_package": m.fs.reaction_params,
                                           "has_heat_of_reaction": True,
                                           "has_heat_transfer": True,
                                           "has_pressure_change": False
                                           })
m.fs.F101 = Flash(default={"property_package": m.fs.thermo_params,
                           "has_heat_transfer": True,
                           "has_pressure_change": True
                           })
m.fs.S101 = Splitter(default={"property_package": m.fs.thermo_params,
                              "ideal_separation": False,
                              "outlet_list": ["purge", "recycle"]
                              })
m.fs.C101 = PressureChanger(default={"property_package": m.fs.thermo_params,
                                     "compressor": True,
                                     "thermodynamic_assumption": ThermodynamicAssumption.isothermal
                                     })
m.fs.F102 = Flash(default={"property_package": m.fs.thermo_params,
                           "has_heat_transfer": True,
                           "has_pressure_change": True
                           })

In [3]:
m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.H101.inlet)
m.fs.s04 = Arc(source=m.fs.H101.outlet, destination=m.fs.R101.inlet)
m.fs.s05 = Arc(source=m.fs.R101.outlet, destination=m.fs.F101.inlet)
m.fs.s06 = Arc(source=m.fs.F101.vap_outlet, destination=m.fs.S101.inlet)
m.fs.s08 = Arc(source=m.fs.S101.recycle, destination=m.fs.C101.inlet)
m.fs.s09 = Arc(source=m.fs.C101.outlet, destination=m.fs.M101.vapor_recycle)
m.fs.s10 = Arc(source=m.fs.F101.liq_outlet, destination=m.fs.F102.inlet)

In [4]:
# Below three lines are a demonstration of the current visualize() function

In [5]:
from idaes.ui.fsvis.fsvis import visualize

In [6]:

visualize(m.fs, 'draftmodel')

 * Serving Flask app "idaes.ui.fsvis.flask_server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5555/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Mar/2020 10:47:06] "POST /fs?id=draftmodel&id=draftmodel HTTP/1.1" 200 -


attempt 0 of 127
Opened in browser window: True
http://127.0.0.1:5555/fs?id=draftmodel


127.0.0.1 - - [18/Mar/2020 10:47:06] "GET /fs?id=draftmodel HTTP/1.1" 200 -


{'model': {'id': 0, 'unit_models': {'M101': {'type': 'mixer', 'image': 'mixer.svg'}, 'H101': {'type': 'heater', 'image': 'heater_2.svg'}, 'R101': {'type': 'stoichiometric_reactor', 'image': 'reactor_s.svg'}, 'F101': {'type': 'flash', 'image': 'flash.svg'}, 'S101': {'type': 'separator', 'image': 'splitter.svg'}, 'C101': {'type': 'pressure_changer', 'image': 'compressor.svg'}, 'F102': {'type': 'flash', 'image': 'flash.svg'}}, 'arcs': {'s03': {'source': 'M101', 'dest': 'H101', 'label': "Molar Flow ('Liq', 'benzene') 0.5\nMolar Flow ('Liq', 'toluene') 0.5\nMolar Flow ('Liq', 'hydrogen') 0.5\nMolar Flow ('Liq', 'methane') 0.5\nMolar Flow ('Vap', 'benzene') 0.5\nMolar Flow ('Vap', 'toluene') 0.5\nMolar Flow ('Vap', 'hydrogen') 0.5\nMolar Flow ('Vap', 'methane') 0.5\nTemperature 298.15\nPressure 10132"}, 's04': {'source': 'H101', 'dest': 'R101', 'label': "Molar Flow ('Liq', 'benzene') 0.5\nMolar Flow ('Liq', 'toluene') 0.5\nMolar Flow ('Liq', 'hydrogen') 0.5\nMolar Flow ('Liq', 'methane') 0.5

127.0.0.1 - - [18/Mar/2020 10:47:13] "POST /fs?id=draftmodel HTTP/1.1" 200 -


{'model': {'unit_models': {'H101': {'type': 'heater', 'image': 'heater_2.svg'}}, 'arcs': {'s03': {'source': 'M101', 'dest': 'H101', 'label': 'Hello World!', 'action': 3, 'class': 'arc'}}, 'id': 0}, 'cells': [{'type': 'standard.Image', 'position': {'x': 200, 'y': 200}, 'size': {'width': 50, 'height': 50}, 'angle': 0, 'id': 'H101', 'z': [1], 'attrs': {'image': {'xlinkHref': 'heater_2.svg'}, 'label': {'text': 'H101'}, 'root': {'title': 'heater'}}}, {'type': 'standard.Link', 'source': {'anchor': {'name': 'right', 'args': {'rotate': 'false', 'padding': 0}}, 'id': 'M101'}, 'target': {'anchor': {'name': 'left', 'args': {'rotate': 'false', 'padding': 0}}, 'id': 'H101'}, 'router': {'name': 'orthogonal', 'padding': 10}, 'connector': {'name': 'normal', 'attrs': {'line': {'stroke': '#5c9adb'}}}, 'id': 's03', 'labels': [{'attrs': {'rect': {'fill': '#d7dce0', 'stroke': '#FFFFFF', 'stroke-width': 1}, 'text': {'text': 'Hello World!', 'fill': 'black', 'text-anchor': 'left'}}, 'position': {'distance': 0

127.0.0.1 - - [18/Mar/2020 10:47:17] "POST /fs?id=draftmodel HTTP/1.1" 200 -


In [ ]:
'''The below is historical work temporarily kept; the entire notebook is to be removed soon anyways'''

In [ ]:
import requests
from requests.exceptions import ConnectionError
import time
import webbrowser

from idaes.ui.fsvis.flask_server import App as fsvis_server

In [ ]:
def visualize(model, modelname):
    server = fsvis_server()
    url = f"http://{server.host}:{server.port}/fs"
    
    repeat_until_connection_available(requests.post, url, json={'model': str(model)}, 
                        params={'id': modelname})
    success = webbrowser.open(url)
    print(f'Opened in browser window: {success}')
    print(f'{url}?id={modelname}')
    return server

def repeat_until_connection_available(f, *args, retries=127, **kwargs):
    for i in range(retries):
        try:
            print(f'attempt {i} of {retries}')
            return f(*args, **kwargs)
        except ConnectionError as e:
            time.sleep(0.1)
            print(f'connection error: attempt {i}; {e}')
            continue # consider logging
            
    # raise ConnectionRefusedError?? how
    # or maybe just print to debug and stop?
        


In [ ]:
s = visualize('fooooo', 'hellllo')

In [ ]:
s.stop() # calling this and then trying to spin up the server again seems to 
    # cause a fairly long delay before the server (re)starts??

In [ ]:
import webbrowser

webbrowser.open("http://127.0.0.1:5555/fs")